In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
url="https://www.cardekho.com/usedCars"

header={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
}

In [8]:
r=requests.get(url)
soup=BeautifulSoup(r.content,"lxml")

In [9]:
parent = soup.find("div",class_="ReactTypeahead  inputfield gs_ta customSelect active")

# city_names=soup.findAll("li",class_="gs_ta_choice")

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import requests

header={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
}
# "Ahmedabad","Banglore","Chennai","Delhi NCR","Gurgaon","Hyderabad","Jaipur","Kolkata","Mumbai",
cities=['Mumbai', 'Pune', 'Chennai', 'Coimbatore', 'Hyderabad', 'Jaipur','Kochi', 'Kolkata', 'Delhi', 'Bangalore', 'Ahmedabad']

In [11]:
car_details=[]

for city in cities:
    
    options=Options()
    driver=webdriver.Chrome(executable_path=r"C:\Users\raksh\Downloads\chromedriver.exe")
    url="https://www.cardekho.com/used-cars+in+{}".format(city)
    driver.get(url)
    
    ScrollNumber = 20
    
    for i in range(1,ScrollNumber):
        print(i)
        driver.execute_script("window.scrollTo(1,50000)")
        time.sleep(1)

    file = open('cardekho.html', 'w',encoding='utf-8')
    file.write(driver.page_source)
    file.close()

    driver.close()
    
    data = open('cardekho.html','r')
    soup = BeautifulSoup(data, 'html.parser')
    
    car_list=soup.find_all("div",class_="bookedbuttonUP width100persent")
    
    car_links=[]
    
    for car in car_list:
        for link in car.find_all("a",href=True):
            car_links.append(link['href'])
    
    
    

    for testlink in car_links:

        r=requests.get(testlink,headers=header)
        soup=BeautifulSoup(r.content,"lxml")

        name=soup.find("h1").text.strip()
        
        price=soup.find("div",class_="priceSection")
        try:
            price=price.text.strip().split(' ')[1].replace(',','')
            if "Fixed" in price:
                price=price.split('F')[0]
        except:
            continue
            
        specifications=soup.find_all("div",class_="listIcons")


        spec=[]
        
        spec.append(name)
        spec.append(city)
        
        f=1
        for details in specifications:


            spec_name=details.find("div",class_="head").text.strip()
            spec_value=details.find("div",class_="fontweight500").text.strip()

            if spec_name=="Make Year":
                spec.append(int(spec_value))
            if spec_name=="KMs Driven":
                spec_value=int(spec_value.split(' ')[0].replace(',',''))
                spec.append(spec_value)
            if spec_name=="Fuel":
                spec.append(spec_value)
            if spec_name=="Transmission":
                spec.append(spec_value)
            if spec_name=="No Of Owner(s)":
                spec.append(spec_value)
            if spec_name=="Mileage":
                spec.append(spec_value)
            if spec_name=="Engine":
                spec.append(spec_value)
            if spec_name=="Max Power" and f:
                f=0
                if "bhp" in spec_value:
                    spec_value=spec_value.split('b')
                    spec.append(float(spec_value[0]))
                else:
                    spec.append("NA")
                
            if spec_name=="Seats":
                spec.append(int(spec_value))

        spec.append(float(price))
        car_details.append(spec)

In [13]:
import csv


columns=["Name","Location","Year","Fuel_Type","Kilometers_Driven","Owner_Type","Transmission","Mileage","Engine","Power","Seats","Price"]


with open('dataset.csv', 'a',encoding='utf-8',newline='') as f: 
    write = csv.writer(f)
    write.writerow(columns)
    write.writerows(car_details)

print("\n\nDone!!\n\n")

